In [4]:
import json
import os
import pandas as pd

In [5]:
fluids = pd.read_csv('fluids.csv')

In [6]:
fluids

,NodeID,FluidType,BufferName,LoadVolume
0,ot2,source,ot2,3
1,2,source,wash buffer,40
2,3,source,imaging buffer,21
3,4,source,tcep,20
4,5,source,2x-SSC,50
5,6,source,universal_fluors,15
6,waste,destination,waste_tank,0


In [56]:
labware = {'tips_200':{
    'api_name': 'opentrons_96_filtertiprack_200ul',
    'location': '1',
    'creator': 'opentrons'
},
              'reservoir1':{
    'api_name': 'nest_12_reservoir_15ml',
    'location': '5',
    'creator': 'opentrons'
                },
              'wellplate1':{
    'api_name': 'nest_96_wellplate_2ml_deep',
    'location': '7',
    'creator': 'opentrons'
                },
              'syringe_adapter':{
    'api_name': 'yeolab_1_bd_syringe_adap_3000ul',
    'location': '10',
    'creator': 'custom'
                },
              }
hardware = {'p300':{
    'api_name': 'p300_single_gen2',
    'mount': 'left',
    'tip_racks': ['tips_200']}}

metadata = {'apiLevel': '2.14'}

ot2_config = {'labware':labware,
              'hardware':hardware,
              'metadata':metadata}

In [79]:
reagents = {}


In [80]:
def generate_well_pos(well_num,total=96,ncol=12,nrow=8):
    row = well_num//ncol
    col = well_num%ncol
    return chr(65+row)+str(col+1)

In [81]:
num_rounds = 2
for i in range(num_rounds):
    reagents['round'+str(i+1)] = {'labware':'wellplate1',
                                'location':generate_well_pos(i),
                                'volume':75,
                           'hardware':'p300',
                           'repeat':2}

In [82]:
def get_fluid_node(buffer):
    return fluids[fluids['BufferName']==buffer]['NodeID'].iloc[0]

In [89]:
experiment = {}

In [90]:
step_n = 0

In [91]:
reagents['round2']

{'labware': 'wellplate1',
 'location': 'A2',
 'volume': 75,
 'hardware': 'p300',
 'repeat': 2}

In [92]:
for round_n in range(1,3):
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'liquid_handler'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['reagent'] = f'round{round_n}'
    experiment[step_n]['step_metadata']['info'] = reagents[f'round{round_n}']
    experiment[step_n]['step_metadata']['destination'] = 'syringe_adapter'
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    """
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('ot2')
    experiment[step_n]['step_metadata']['volume'] = 1.5
    experiment[step_n]['step_metadata']['speed'] = 1
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    """
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 10 # 15 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1

In [93]:
experiment

{0: {'step_type': 'liquid_handler',
  'step_metadata': {'reagent': 'round1',
   'info': {'labware': 'wellplate1',
    'location': 'A1',
    'volume': 75,
    'hardware': 'p300',
    'repeat': 2},
   'destination': 'syringe_adapter'},
  'slack_notify': False},
 1: {'step_type': 'wait',
  'step_metadata': {'wait_time': 10},
  'slack_notify': False},
 2: {'step_type': 'liquid_handler',
  'step_metadata': {'reagent': 'round2',
   'info': {'labware': 'wellplate1',
    'location': 'A2',
    'volume': 75,
    'hardware': 'p300',
    'repeat': 2},
   'destination': 'syringe_adapter'},
  'slack_notify': False},
 3: {'step_type': 'wait',
  'step_metadata': {'wait_time': 10},
  'slack_notify': False}}

In [88]:
with open('experiment.json', 'w+') as f:
    json.dump(experiment, f, indent=4)


In [78]:
with open('ot2_config.json', 'w+') as f:
    json.dump(ot2_config, f, indent=4)


In [34]:
def get_tip_num(experiment,step_num):
    # figure out which steps are liquid handling rounds
    liquid_handling_steps = []
    for step, value in experiment.items():
        if value['step_type'] == 'liquid_handler':
            liquid_handling_steps.append(step)
    tip_num = liquid_handling_steps.index(step_num)
    return tip_num

In [35]:
get_tip_num(experiment,0)

0